In [1]:
import geopandas as gpd
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import LineString, Point
import json
import os

In [2]:
def convert_to_geodataframe_points(df):
    # Create a copy of the dataframe to avoid modifying the original
    gdf = df.copy()
    
    # Convert the coordinates to shapely Point objects
    gdf['geometry'] = gdf['coords'].apply(lambda x: Point(x[0], x[1]))  # Note: Point(lat, lon)
    
    # Create the GeoDataFrame
    gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
    
    # Set the coordinate reference system (CRS) to WGS84
    gdf.set_crs(epsg=4326, inplace=True)
    # drop coords column
    gdf = gdf.drop(columns=['coords'])
    #rename index column
    gdf.rename(columns={"index": "bridge_index"}, inplace=True)
    
    return gdf

In [5]:
def drop_unused_columns(df):
    # drop columns that are not needed
    drop_cols = ["type", "geometry", "subregion_indices"]
    for c in df.columns:
        if c.startswith("used_by_cells"):
            drop_cols.append(c)
    for c in df.columns:
        if "_fixed" in c:
            drop_cols.append(c)
    return df.drop(columns=drop_cols)

In [25]:
def convert_array_columns(df):
    df_fixed = df.copy()
    print(f"Starting conversion process for {len(df.columns)} columns")

    # Loop through each column
    complex_cols_found = 0
    for i, col in enumerate(df.columns):
        print(f"Processing column {i+1}/{len(df.columns)}: '{col}'")
        
        # Check if column contains complex data types
        has_complex = df[col].apply(lambda x: isinstance(x, (np.ndarray, list, dict))).any()
        
        if has_complex:
            complex_cols_found += 1
            print(f"  Found complex data in column '{col}' - converting...")
            
            # Sample the first complex value for debugging
            sample_idx = df[col].apply(lambda x: isinstance(x, (np.ndarray, list, dict))).idxmax()
            sample_val = df.loc[sample_idx, col]
            print(f"  Sample value before conversion: {type(sample_val)}, first few items: {str(sample_val)[:50]}...")
            
            # Convert complex values to JSON strings
            df_fixed[col] = df[col].apply(
                lambda x: json.dumps(x.tolist() if isinstance(x, np.ndarray) else x) 
                if isinstance(x, (np.ndarray, list, dict)) else x
            )
            
            # Verify conversion worked
            new_sample_val = df_fixed.loc[sample_idx, col]
            print(f"  Sample value after conversion: {type(new_sample_val)}, preview: {str(new_sample_val)[:50]}...")

    print(f"Conversion complete! Found and converted {complex_cols_found} columns with complex data types")
    
    return df_fixed

In [26]:
def process_data(file_path):
    # Load the data
    df = pd.read_parquet(file_path)
    print(f"Loaded data with shape: {df.shape}")

     # Drop unused columns
    df = drop_unused_columns(df)
    print(f"Dropped unused columns, new shape: {df.shape}")

    # Convert the data to a GeoDataFrame
    df = convert_to_geodataframe_points(df)
    print(f"Converted to GeoDataFrame with shape: {df.shape}")

    # Convert array columns to JSON strings
    df = convert_array_columns(df)
    print(f"Converted columns. New shape: {df.shape}")

    return df

In [2]:
# paths
local_path = "/Volumes/samsung-4tb/b2p/impact-model/no_order_1_less_than_500m_with_top_sites/model_outputs/"
bridge_view_path = "joined_data/bridge_view_data.parquet"
output_folder_path ="/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/"

et_path = os.path.join(local_path, "ethiopia/",bridge_view_path)
civ_path = os.path.join(local_path, "ivory_coast/",bridge_view_path)
kenya_path = os.path.join(local_path, "kenya/",bridge_view_path)
rwanda_path = os.path.join(local_path, "rwanda/",bridge_view_path)
tanzania_path = os.path.join(local_path, "tanzania/",bridge_view_path)
uganda_path = os.path.join(local_path, "uganda/",bridge_view_path)
zambia_path = os.path.join(local_path, "zambia/",bridge_view_path)


In [44]:
test = pd.read_parquet(et_path)
test

,index,type,geometry,subregion_indices,coords,exit_point_index,used_by_cells_for_semi_dense_urban_optimal,used_by_h3_for_semi_dense_urban_optimal,used_by_cells_for_health_posts_optimal,used_by_h3_for_health_posts_optimal,used_by_cells_for_primary_schools_fixed,used_by_h3_for_primary_schools_fixed,used_by_cells_for_all_health_facilities_optimal,used_by_h3_for_all_health_facilities_optimal,used_by_cells_for_health_centers_optimal,used_by_h3_for_health_centers_optimal,used_by_cells_for_major_hospitals_optimal,used_by_h3_for_major_hospitals_optimal,used_by_cells_for_major_roads_optimal,used_by_h3_for_major_roads_optimal,used_by_cells_for_secondary_schools_fixed,used_by_h3_for_secondary_schools_fixed,used_by_cells_for_all_education_facilities_fixed,used_by_h3_for_all_education_facilities_fixed
0,302351,bridge_predicted,"b'\x01\x01\x00\x00\x00I\xa2\x97Q,\xbbB@.\x90\x...","[269620, 269629]","[37.46229, 4.8498]",302352,"[[12062, 5346], [12063, 5345], [12064, 5344], ...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...","[[12062, 5346], [12063, 5345], [12064, 5344], ...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...",[],[],"[[12060, 5355], [12061, 5356], [12059, 5355], ...","[886a4b39a7fffff, 886a4b39a7fffff, 886a4b39a7f...","[[12062, 5346], [12063, 5345], [12064, 5344], ...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...",[],[],[],[],[],[],[],[]
1,302352,bridge_predicted,b'\x01\x01\x00\x00\x00\x1d\xc9\xe5?\xa4\xbbB@\...,"[269620, 270228]","[37.46595, 4.85105]",302353,"[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12044, 5353], [12045, 5354], [12046, 5355], ...","[886a4b39a1fffff, 886a4b39a1fffff, 886a4b39a1f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],[],[],[],[],[],[]
2,302353,bridge_predicted,b'\x01\x01\x00\x00\x00\x08\x03\xcf\xbd\x87\xbb...,"[269906, 270228]","[37.46508, 4.85778]",302354,"[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12050, 5358], [12039, 5351], [12040, 5352], ...","[886a4b39a1fffff, 886a4b39a9fffff, 886a4b39a9f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],[],[],[],[],[],[]
3,302354,bridge_predicted,b'\x01\x01\x00\x00\x00\x91\xb8\xc7\xd2\x87\xba...,"[267568, 269906]","[37.45727, 4.86828]",302355,"[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12037, 5349], [12038, 5350]]","[886a4b39e7fffff, 886a4b39a9fffff]","[[12099, 5344], [12099, 5345], [12099, 5346], ...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[],[],"[[12037, 5349], [12038, 5350], [12039, 5351], ...","[886a4b39e7fffff, 886a4b39a9fffff, 886a4b39a9f...",[],[],[],[]
4,302355,bridge_predicted,"b'\x01\x01\x00\x00\x00\x18>""\xa6D\xbaB@%;6\x02...","[267568, 269717]","[37.45522, 4.87006]",302356,"[[12028, 5319], [12029, 5318], [12030, 5317], ...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...","[[12028, 5319], [12029, 5318], [12030, 5317], ...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...",[],[],"[[12032, 5326], [12033, 5325], [12034, 5324], ...","[886a4b76d9fffff, 886a4b76d9fffff, 886a4b76d9f...","[[12028, 5319], [12029, 5318], [12030, 5317], ...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...",[],[],"[[12034, 5330], [12035, 5330], [12035, 5339], ...","[886a4b76d9fffff, 886a4b76d9fffff, 886a4b76dbf...",[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46241,348592,bridge_predicted,b'\x01\x01\x00\x00\x00_F\xb1\xdc\xd2\x1eC@\xe3...,"[34

In [30]:
et_bridges = process_data(et_path)

Loaded data with shape: (46246, 24)
Dropped unused columns, new shape: (46246, 9)
Converted to GeoDataFrame with shape: (46246, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['886a4b769bfffff' '886a4b769bfffff' '886a4b7691ff...
  Sample value after conversion: <class 'str'>, preview: ["886a4b769bfffff", "886a4b769bfffff", "886a4b7691...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['886a4b769bfffff' '886a4b769bfffff' '886a4b7691ff...
  Sample value after conversion: <class 'str'>, preview: ["886a4b769bfffff",

In [31]:
civ_bridges = process_data(civ_path)

Loaded data with shape: (9048, 24)
Dropped unused columns, new shape: (9048, 9)
Converted to GeoDataFrame with shape: (9048, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['8874368f33fffff' '8874368f33fffff']...
  Sample value after conversion: <class 'str'>, preview: ["8874368f33fffff", "8874368f33fffff"]...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['8874368dd9fffff' '8874368ce5fffff' '8874368ce5ff...
  Sample value after conversion: <class 'str'>, preview: ["8874368dd9fffff", "8874368ce5fffff", "8874368

In [32]:
kenya_bridges = process_data(kenya_path)

Loaded data with shape: (13191, 24)
Dropped unused columns, new shape: (13191, 9)
Converted to GeoDataFrame with shape: (13191, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['887b4a0633fffff']...
  Sample value after conversion: <class 'str'>, preview: ["887b4a0633fffff"]...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['887b4a0633fffff' '887b4a0623fffff' '887b4a0621ff...
  Sample value after conversion: <class 'str'>, preview: ["887b4a0633fffff", "887b4a0623fffff", "887b4a0621...
Processing column 5/9: 'use

In [33]:
rwanda_bridges = process_data(rwanda_path)

Loaded data with shape: (1010, 24)
Dropped unused columns, new shape: (1010, 9)
Converted to GeoDataFrame with shape: (1010, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['886ada8f0bfffff' '886ada8f09fffff' '886ada8f09ff...
  Sample value after conversion: <class 'str'>, preview: ["886ada8f0bfffff", "886ada8f09fffff", "886ada8f09...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['886ada8f0bfffff' '886ada8f09fffff' '886ada8f09ff...
  Sample value after conversion: <class 'str'>, preview: ["886ada8f0bfffff", "8

In [34]:
tanzania_bridges = process_data(tanzania_path)

Loaded data with shape: (25616, 24)
Dropped unused columns, new shape: (25616, 9)
Converted to GeoDataFrame with shape: (25616, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['8896a471d1fffff' '8896a471d1fffff' '8896a471d1ff...
  Sample value after conversion: <class 'str'>, preview: ["8896a471d1fffff", "8896a471d1fffff", "8896a471d1...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['8896a47189fffff' '8896a47189fffff' '8896a47189ff...
  Sample value after conversion: <class 'str'>, preview: ["8896a47189fffff",

In [35]:
uganda_bridges = process_data(uganda_path)

Loaded data with shape: (5951, 24)
Dropped unused columns, new shape: (5951, 9)
Converted to GeoDataFrame with shape: (5951, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['886adc2951fffff' '886adc2951fffff' '886adc2951ff...
  Sample value after conversion: <class 'str'>, preview: ["886adc2951fffff", "886adc2951fffff", "886adc2951...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['886adc2aedfffff' '886adc2aedfffff' '886adc2aedff...
  Sample value after conversion: <class 'str'>, preview: ["886adc2aedfffff", "8

In [36]:
zambia_bridges = process_data(zambia_path)

Loaded data with shape: (15212, 24)
Dropped unused columns, new shape: (15212, 9)
Converted to GeoDataFrame with shape: (15212, 9)
Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
  Found complex data in column 'used_by_h3_for_semi_dense_urban_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['88975140ebfffff']...
  Sample value after conversion: <class 'str'>, preview: ["88975140ebfffff"]...
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
  Found complex data in column 'used_by_h3_for_health_posts_optimal' - converting...
  Sample value before conversion: <class 'numpy.ndarray'>, first few items: ['88975140e1fffff' '88975140e1fffff' '88975140e1ff...
  Sample value after conversion: <class 'str'>, preview: ["88975140e1fffff", "88975140e1fffff", "88975140e1...
Processing column 5/9: 'use

In [37]:
# join all geodataframes
all_bridges = pd.concat([et_bridges, civ_bridges, kenya_bridges, rwanda_bridges, tanzania_bridges, uganda_bridges, zambia_bridges])

In [38]:
all_bridges

,bridge_index,exit_point_index,used_by_h3_for_semi_dense_urban_optimal,used_by_h3_for_health_posts_optimal,used_by_h3_for_all_health_facilities_optimal,used_by_h3_for_health_centers_optimal,used_by_h3_for_major_hospitals_optimal,used_by_h3_for_major_roads_optimal,geometry
0,302351,302352,"[""886a4b769bfffff"", ""886a4b769bfffff"", ""886a4b...","[""886a4b769bfffff"", ""886a4b769bfffff"", ""886a4b...","[""886a4b39a7fffff"", ""886a4b39a7fffff"", ""886a4b...","[""886a4b769bfffff"", ""886a4b769bfffff"", ""886a4b...",[],[],POINT (37.46229 4.84980)
1,302352,302353,"[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...","[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...","[""886a4b39a1fffff"", ""886a4b39a1fffff"", ""886a4b...","[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...",[],[],POINT (37.46595 4.85105)
2,302353,302354,"[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...","[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...","[""886a4b39a1fffff"", ""886a4b39a9fffff"", ""886a4b...","[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...",[],[],POINT (37.46508 4.85778)
3,302354,302355,"[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...","[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...","[""886a4b39e7fffff"", ""886a4b39a9fffff""]","[""886a4b2b65fffff"", ""886a4b2b65fffff"", ""886a4b...",[],"[""886a4b39e7fffff"", ""886a4b39a9fffff"", ""886a4b...",POINT (37.45727 4.86828)
4,302355,302356,"[""886a4b3937fffff"", ""886a4b3937fffff"", ""886a4b...","[""886a4b3937fffff"", ""886a4b3937fffff"", ""886a4b...","[""886a4b76d9fffff"", ""886a4b76d9fffff"", ""886a4b...","[""886a4b3937fffff"", ""886a4b3937fffff"", ""886a4b...",[],"[""886a4b76d9fffff"", ""886a4b76d9fffff"", ""886a4b...",POINT (37.45522 4.87006)
...,...,...,...,...,...,...,...,...,...
15207,105971,105972,"[""8896a9a269fffff"", ""8896a9b197fffff"", ""8896a9...","[""8896a9a269fffff"", ""8896a9b197fffff"", ""8896a9...","[""8896a9a269fffff"", ""8896a9b197fffff"", ""8896a9...","[""8896a9a269fffff"", ""8896a9b197fffff"", ""8896a9...","[""8896a9a269fffff"", ""8896a9b197fffff"", ""8896a9...",[],POINT (29.63156 -8.49877)
15208,105972,105973,"[""8896a9a345fffff"", ""8896a9a345fffff"", ""8896a9...","[""8896a9a225fffff"", ""8896a9a225fffff"", ""8896a9...","[""8896a9a345fffff"", ""8896a9a345fffff"", ""8896a9...","[""8896a9a345fffff"", ""8896a9a345fffff"", ""8896a9...","[""8896a9a225fffff"", ""8896a9a225fffff"", ""8896a9...",[],POINT (29.63656 -8.48563)
15209,105973,105974,"[""8896f4d2d3fffff"", ""8896f4d2d3fffff"", ""8896f4...","[""8896f4d2d3fffff"", ""8896f4d2d3fffff"", ""8896f4...","[""8896f4d287fffff"", ""8896f4d287fffff"", ""8896f4...","[""8896f4d287fffff"", ""8896f4d287fffff"", ""8896f4...","[""8896f4d287fffff"", ""8896f4d287fffff"", ""8896f4...",[],POINT (29.11544 -8.47639)
15210,105974,105975,"[""8896a98cb3fffff"", ""8896a98cb3fffff"", ""8896a9...","[""8896a9b91dfffff"", ""8896a9b91dfffff"", ""8896a9...","[""8896a9b91dfffff"", ""8896a9b91dfffff"", ""8896a9...","[""8896a9b90dfffff"", ""8896a9b96bfffff"", ""8896a9...","[""8896a98cb3fffff"", ""8896a98cb3fffff"", ""8896a9...",[],POINT (29.75404 -8.41852)


In [39]:
# write each individual country to geojson
et_bridges.to_file(os.path.join(output_folder_path, "ethiopia_bridges.geojson"), driver='GeoJSON')
print ("Ethiopia bridges written to geojson")
civ_bridges.to_file(os.path.join(output_folder_path, "ivory_coast_bridges.geojson"), driver='GeoJSON')
print("Ivory Coast bridges written to geojson")
kenya_bridges.to_file(os.path.join(output_folder_path, "kenya_bridges.geojson"), driver='GeoJSON')
print("Kenya bridges written to geojson")
rwanda_bridges.to_file(os.path.join(output_folder_path, "rwanda_bridges.geojson"), driver='GeoJSON')
print("Rwanda bridges written to geojson")
tanzania_bridges.to_file(os.path.join(output_folder_path, "tanzania_bridges.geojson"), driver='GeoJSON')
print("Tanzania bridges written to geojson")
uganda_bridges.to_file(os.path.join(output_folder_path, "uganda_bridges.geojson"), driver='GeoJSON')
print("Uganda bridges written to geojson")
zambia_bridges.to_file(os.path.join(output_folder_path, "zambia_bridges.geojson"), driver='GeoJSON')
print("Zambia bridges written to geojson")

Ethiopia bridges written to geojson
Ivory Coast bridges written to geojson
Kenya bridges written to geojson
Rwanda bridges written to geojson
Tanzania bridges written to geojson
Uganda bridges written to geojson
Zambia bridges written to geojson


In [40]:
# write to geojson
all_bridges = convert_array_columns(all_bridges)
all_bridges = gpd.GeoDataFrame(all_bridges, geometry='geometry')
all_bridges.set_crs(epsg=4326, inplace=True)
all_bridges.to_file(os.path.join(output_folder_path, "all_bridges.geojson"), driver='GeoJSON')
print("Done!")

Starting conversion process for 9 columns
Processing column 1/9: 'bridge_index'
Processing column 2/9: 'exit_point_index'
Processing column 3/9: 'used_by_h3_for_semi_dense_urban_optimal'
Processing column 4/9: 'used_by_h3_for_health_posts_optimal'
Processing column 5/9: 'used_by_h3_for_all_health_facilities_optimal'
Processing column 6/9: 'used_by_h3_for_health_centers_optimal'
Processing column 7/9: 'used_by_h3_for_major_hospitals_optimal'
Processing column 8/9: 'used_by_h3_for_major_roads_optimal'
Processing column 9/9: 'geometry'
Conversion complete! Found and converted 0 columns with complex data types
Done!


## Creating Small Bridges Dataset

In [3]:
all_bridges = gpd.read_file(os.path.join(output_folder_path, "all_bridges.geojson"))
all_bridges

,bridge_index,exit_point_index,used_by_h3_for_semi_dense_urban_optimal,used_by_h3_for_health_posts_optimal,used_by_h3_for_all_health_facilities_optimal,used_by_h3_for_health_centers_optimal,used_by_h3_for_major_hospitals_optimal,used_by_h3_for_major_roads_optimal,geometry
0,302351,302352,"[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...","[886a4b39a7fffff, 886a4b39a7fffff, 886a4b39a7f...","[886a4b769bfffff, 886a4b769bfffff, 886a4b7691f...",[ ],[ ],POINT (37.46229 4.84980)
1,302352,302353,"[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[886a4b39a1fffff, 886a4b39a1fffff, 886a4b39a1f...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[ ],[ ],POINT (37.46595 4.85105)
2,302353,302354,"[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[886a4b39a1fffff, 886a4b39a9fffff, 886a4b39a9f...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[ ],[ ],POINT (37.46508 4.85778)
3,302354,302355,"[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...","[886a4b39e7fffff, 886a4b39a9fffff]","[886a4b2b65fffff, 886a4b2b65fffff, 886a4b2b65f...",[ ],"[ ""886a4b39e7fffff"", ""886a4b39a9fffff"", ""886a4...",POINT (37.45727 4.86828)
4,302355,302356,"[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...","[886a4b76d9fffff, 886a4b76d9fffff, 886a4b76d9f...","[886a4b3937fffff, 886a4b3937fffff, 886a4b3937f...",[ ],"[ ""886a4b76d9fffff"", ""886a4b76d9fffff"", ""886a4...",POINT (37.45522 4.87006)
...,...,...,...,...,...,...,...,...,...
116269,105971,105972,"[8896a9a269fffff, 8896a9b197fffff, 8896a9b197f...","[8896a9a269fffff, 8896a9b197fffff, 8896a9b197f...","[8896a9a269fffff, 8896a9b197fffff, 8896a9b197f...","[8896a9a269fffff, 8896a9b197fffff, 8896a9b197f...","[ ""8896a9a269fffff"", ""8896a9b197fffff"", ""8896a...",[ ],POINT (29.63156 -8.49877)
116270,105972,105973,"[8896a9a345fffff, 8896a9a345fffff, 8896a9a345f...","[8896a9a225fffff, 8896a9a225fffff, 8896a9a225f...","[8896a9a345fffff, 8896a9a345fffff, 8896a9a345f...","[8896a9a345fffff, 8896a9a345fffff, 8896a9a345f...","[ ""8896a9a225fffff"", ""8896a9a225fffff"", ""8896a...",[ ],POINT (29.63656 -8.48563)
116271,105973,105974,"[8896f4d2d3fffff, 8896f4d2d3fffff, 8896f4d2d3f...","[8896f4d2d3fffff, 8896f4d2d3fffff, 8896f4d2d3f...","[8896f4d287fffff, 8896f4d287fffff, 8896f4d287f...","[8896f4d287fffff, 8896f4d287fffff, 8896f4d287f...","[ ""8896f4d287fffff"", ""8896f4d287fffff"", ""8896f...",[ ],POINT (29.11544 -8.47639)
116272,105974,105975,"[8896a98cb3fffff, 8896a98cb3fffff, 8896a98cb3f...","[8896a9b91dfffff, 8896a9b91dfffff, 8896a9b91df...","[8896a9b91dfffff, 8896a9b91dfffff, 8896a9b91df...","[8896a9b90dfffff, 8896a9b96bfffff, 8896a9b96bf...","[ ""8896a98cb3fffff"", ""8896a98cb3fffff"", ""8896a...",[ ],POINT (29.75404 -8.41852)


In [4]:
for c in all_bridges.columns:
    print(c)

bridge_index
exit_point_index
used_by_h3_for_semi_dense_urban_optimal
used_by_h3_for_health_posts_optimal
used_by_h3_for_all_health_facilities_optimal
used_by_h3_for_health_centers_optimal
used_by_h3_for_major_hospitals_optimal
used_by_h3_for_major_roads_optimal
geometry


In [5]:
# drop all columns except geometry and bridge_index
all_bridges_tiny = all_bridges[["bridge_index", "geometry"]]
all_bridges_tiny.to_file(os.path.join(output_folder_path, "all_bridges_tiny.geojson"), driver='GeoJSON')
print("Done!")

Done!


## Tileset Development

In [31]:
tiny_bridges_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges_tiny.geojson"
tiny_bridges_tiles_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges_tiny_tiles.mbtiles"
full_bridges_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges.geojson"
full_bridges_tiles_path = "/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/full_bridges_tiles2.mbtiles"


In [19]:
print(tiny_bridges_path)
print (tiny_bridges_tiles_path)

/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges_tiny.geojson
/Volumes/samsung-4tb/b2p/impact-model/cleaned_data/all_bridges_tiny_tiles.mbtiles


In [27]:
!tippecanoe -o {tiny_bridges_tiles_path} -r1 -zg --drop-densest-as-needed --extend-zooms-if-still-dropping {tiny_bridges_path} --force

For layer 0, using name "all_bridges_tiny"
116274 features, 3969330 bytes of geometry and attributes, 875144 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z6 for features typically 6552 feet (1997 meters) apart, and at least 1376 feet (420 meters) apart
tile 0/0/0 size is 715960 (probably really 715960) with detail 12, >500000    
Going to try keeping the sparsest 55.87% of the features to make it fit
tile 4/9/7 size is 502034 (probably really 502034) with detail 12, >500000    
Going to try keeping the sparsest 79.68% of the features to make it fit
  97.2%  6/38/30  


In [ ]:
!tippecanoe -o {full_bridges_tiles_path} -r1 -zg --drop-densest-as-needed --extend-zooms-if-still-dropping --drop-densest-as-needed --no-feature-limit --maximum-zoom=g {full_bridges_path} --force

tippecanoe: unrecognized option `--extent-zooms-if-still-dropping'
Usage: tippecanoe [options] [file.json ...]
  Output tileset
         --output=output.mbtiles [--output-to-directory=...] [--force]
         [--allow-existing]
  Tileset description and attribution
         [--name=...] [--attribution=...] [--description=...]
  Input files and layer names
         [--layer=...] [--named-layer=...]
  Parallel processing of input
         [--read-parallel]
  Projection of input
         [--projection=...]
  Zoom levels
         [--maximum-zoom=...] [--minimum-zoom=...]
         [--smallest-maximum-zoom-guess=...]
         [--extend-zooms-if-still-dropping]
         [--extend-zooms-if-still-dropping-maximum=...]
         [--generate-variable-depth-tile-pyramid] [--one-tile=...]
  Tile resolution
         [--full-detail=...] [--low-detail=...] [--minimum-detail=...]
         [--extra-detail=...]
  Filtering feature attributes
         [--exclude=...] [--include=...] [--exclude-all]
  Modify

In [30]:
!tippecanoe -o {full_bridges_tiles_path} -r1 -zg --limit-base-zoom-to-maximum-zoom {full_bridges_path} --force

For layer 0, using name "all_bridges"
116274 features, 8313055 bytes of geometry and attributes, 7638104223 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z6 for features typically 6552 feet (1997 meters) apart, and at least 1376 feet (420 meters) apart
tile 0/0/0 size is 96721081 (probably really 96721081) with detail 12, >500000    
tile 0/0/0 size is 96721080 (probably really 96721080) with detail 11, >500000    
tile 0/0/0 size is 96721080 (probably really 96721080) with detail 10, >500000    
tile 0/0/0 size is 96721080 (probably really 96721080) with detail 9, >500000    
tile 0/0/0 size is 96721081 (probably really 96721081) with detail 8, >500000    
tile 0/0/0 size is 96718576 (probably really 96718576) with detail 7, >500000    
could not make tile 0/0/0 small enough
tippecanoe: No zoom levels were successfully written


In [35]:
!tippecanoe -o {full_bridges_tiles_path} --base-zoom=10 -r1 {full_bridges_path} --force


For layer 0, using name "all_bridges"
116274 features, 8133925 bytes of geometry and attributes, 7638104223 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
tile 0/0/0 size is 96721081 (probably really 96721081) with detail 12, >500000    
tile 0/0/0 size is 96721080 (probably really 96721080) with detail 11, >500000    
tile 0/0/0 size is 96721080 (probably really 96721080) with detail 10, >500000    
tile 0/0/0 size is 96721080 (probably really 96721080) with detail 9, >500000    
tile 0/0/0 size is 96721081 (probably really 96721081) with detail 8, >500000    
tile 0/0/0 size is 96718576 (probably really 96718576) with detail 7, >500000    
could not make tile 0/0/0 small enough
tippecanoe: No zoom levels were successfully written


In [ ]:
!tippecanoe -o {full_bridges_tiles_path} -pf -z12 -Z12 -r1 --extend-zooms-if-still-dropping {full_bridges_path} --force


For layer 0, using name "all_bridges"
116274 features, 8313055 bytes of geometry and attributes, 7638104223 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
